In [1]:
import requests
import zipfile
import re
import os
import glob
import pandas as pd

In [2]:
confirmed = requests.get('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv', stream=True)
deaths = requests.get('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv', stream=True)
recovered = requests.get('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv', stream=True)

In [3]:
prefix = 'datasets/'
with open(prefix + 'confirmed.csv', 'w') as file:
    file.write(confirmed.text)
with open(prefix + 'deaths.csv', 'w') as file:
    file.write(deaths.text)
with open(prefix + 'recovered.csv', 'w') as file:
    file.write(recovered.text)

For merging the historical data files

In [2]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [3]:
download_url('https://github.com/CSSEGISandData/COVID-19/archive/master.zip', 'temp/coronadata.zip')

In [4]:
z = zipfile.ZipFile('temp/coronadata.zip')

In [5]:
selector = re.compile('\d+-\d+-\d+')
files = []
for file in z.filelist:
    if ('csse_covid_19_daily_reports' in file.filename and selector.search(file.filename)):
        files.append(file.filename)

In [7]:
for file in z.filelist:
    if (file.filename in files):
        print(file.filename)
        if zip_info.filename[-1] == '/':
            continue
        old_path = file.filename
        file.filename = os.path.basename(file.filename)
        z.extract(old_path, 'temp/')

COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv


NameError: name 'zip_info' is not defined

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('temp/*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "datasets/corona_data.csv", index=False, encoding='utf-8-sig')